In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [1]:
!ls

adc.json  drive  sample_data


In [2]:
cd drive/Houston/

/content/drive/Houston


In [0]:
import numpy as np
import cv2

In [0]:
area_hsi = []
for i in range(3):
    for j in range(14):
        img_path = "area/pc_area_" + str(i) + "_" + str(j) + ".png"
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        area_hsi.append(img)
        
area_hsi = np.array(area_hsi)

In [0]:
area_lidar = []
for i in range(14):
    img_path = "area/lidar_area_" + str(i) + ".png"
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    area_lidar.append(img)

area_lidar = np.array(area_lidar)

In [6]:
print(area_hsi.shape)
print(area_lidar.shape)

(42, 349, 1905)
(14, 349, 1905)


In [0]:
train_file_name = "labels/train.txt"
test_file_name = "labels/test.txt"

In [0]:
file = open(train_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
train_array = np.array(triplets, dtype=int)
file.close()


file = open(test_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
test_array = np.array(triplets, dtype=int)
file.close()

HEIGHT = train_array.shape[0]
WIDTH = train_array.shape[1]

In [0]:
area_hsi_train_data = []
area_hsi_test_data = []
area_lidar_train_data = []
area_lidar_test_data = []
moment_hsi_train_data = []
moment_hsi_test_data = []
moment_lidar_train_data = []
moment_lidar_test_data = []
train_labels = []
test_labels = []
for i in range(HEIGHT):
    for j in range(WIDTH):
        if train_array[i, j] != 0:
            area_hsi_train_data.append(area_hsi[:, i, j])
            area_lidar_train_data.append(area_lidar[:, i, j])
            train_labels.append(train_array[i, j])
        if test_array[i,j] != 0:
            area_hsi_test_data.append(area_hsi[:, i, j])
            area_lidar_test_data.append(area_lidar[:, i, j])
            test_labels.append(test_array[i, j])

In [0]:
area_hsi_train_data = np.array(area_hsi_train_data)
area_hsi_test_data = np.array(area_hsi_test_data)
area_lidar_train_data = np.array(area_lidar_train_data)
area_lidar_test_data = np.array(area_lidar_test_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [11]:
print(area_hsi_train_data.shape)
print(area_hsi_test_data.shape)
print(area_lidar_train_data.shape)
print(area_lidar_test_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(2832, 42)
(12197, 42)
(2832, 14)
(12197, 14)
(2832,)
(12197,)


In [12]:
import keras

train_one_hot = keras.utils.to_categorical(train_labels-1)
test_one_hot = keras.utils.to_categorical(test_labels-1)

Using TensorFlow backend.


In [13]:
print(train_one_hot.shape)
print(test_one_hot.shape)

(2832, 15)
(12197, 15)


In [0]:
HSI_PATCH_SIZE = 27
LiDAR_PATCH_SIZE = 41
CONV1 = 500
CONV2 = 100
FC1 = 200
FC2 = 84
LEARNING_RATE = 0.005
BATCH_SIZE = 25

In [0]:
import numpy as np
padded_area_hsi = np.lib.pad(area_hsi, ((0,0), (HSI_PATCH_SIZE//2, HSI_PATCH_SIZE//2), (HSI_PATCH_SIZE//2,HSI_PATCH_SIZE//2)), 'reflect')
padded_area_lidar = np.lib.pad(area_lidar, ((0,0), (LiDAR_PATCH_SIZE//2, LiDAR_PATCH_SIZE//2), (LiDAR_PATCH_SIZE//2,LiDAR_PATCH_SIZE//2)), 'reflect')

In [16]:
print(padded_area_hsi.shape)
print(padded_area_lidar.shape)

(42, 375, 1931)
(14, 389, 1945)


In [0]:
def get_patches(data, patch_size, row, column):
  offset = patch_size // 2
  row_low = row - offset
  row_high = row + offset
  col_low = column - offset
  col_high = column + offset
  return data[0:, row_low:row_high + 1, col_low:col_high + 1].reshape(patch_size, patch_size, data.shape[0])

In [0]:
area_hsi_train_patches = []
area_hsi_test_patches = []
area_lidar_train_patches = []
area_lidar_test_patches = []

for i in range(HEIGHT):
  for j in range(WIDTH):
    if train_array[i, j] != 0:
      area_hsi_train_patches.append(get_patches(padded_area_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))
      area_lidar_train_patches.append(get_patches(padded_area_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))
    if test_array[i, j] != 0:
      area_hsi_test_patches.append(get_patches(padded_area_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))
      area_lidar_test_patches.append(get_patches(padded_area_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))

In [0]:
area_hsi_train_patches = np.array(area_hsi_train_patches)
area_hsi_test_patches = np.array(area_hsi_test_patches)
area_lidar_train_patches = np.array(area_lidar_train_patches)
area_lidar_test_patches = np.array(area_lidar_test_patches)

In [20]:
print(area_hsi_train_patches.shape)
print(area_hsi_test_patches.shape)
print(area_lidar_train_patches.shape)
print(area_lidar_test_patches.shape)

(2832, 27, 27, 42)
(12197, 27, 27, 42)
(2832, 41, 41, 14)
(12197, 41, 41, 14)


In [0]:
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = area_hsi_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]


hsi_input = Input(shape=(HSI_PATCH_SIZE, HSI_PATCH_SIZE, BANDS))

hsi_conv1 = Conv2D(kernel_size=3, strides=1, filters=CONV1, padding='same', activation='relu', name='hsi_conv1')(hsi_input)
hsi_batch1 = BatchNormalization()(hsi_conv1)
hsi_max_pool1 = MaxPooling2D(pool_size=2, strides=2)(hsi_batch1)
hsi_conv2 = Conv2D(kernel_size=5, strides=1, filters=CONV2, padding='same', activation='relu', name='hsi_conv2')(hsi_max_pool1)
hsi_batch2 = BatchNormalization()(hsi_conv2)
hsi_max_pool2 = MaxPooling2D(pool_size=2, strides=2)(hsi_batch2)

hsi_flatten = Flatten()(hsi_max_pool2)

In [0]:
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer, Input, concatenate
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = area_lidar_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]


lidar_input = Input(shape=(LiDAR_PATCH_SIZE, LiDAR_PATCH_SIZE, BANDS))

lidar_conv1 = Conv2D(kernel_size=3, strides=1, filters=CONV1, padding='same', activation='relu', name='lidar_conv1')(lidar_input)
lidar_batch1 = BatchNormalization()(lidar_conv1)
lidar_max_pool1 = MaxPooling2D(pool_size=2, strides=2)(lidar_batch1)
lidar_conv2 = Conv2D(kernel_size=3, strides=1, filters=CONV2, padding='same', activation='relu', name='lidar_conv2')(lidar_max_pool1)
lidar_batch2 = BatchNormalization()(lidar_conv2)
lidar_max_pool2 = MaxPooling2D(pool_size=2, strides=2)(lidar_batch2)

lidar_flatten = Flatten()(lidar_max_pool2)

In [0]:
fusion_input = concatenate([hsi_flatten, lidar_flatten])

In [0]:
fusion_fc1 = Dense(FC1, activation='relu')(fusion_input)
fusion_drop1 = Dropout(0.5)(fusion_fc1)
fusion_fc2 = Dense(FC1, activation='relu')(fusion_drop1)
fusion_drop2 = Dropout(0.1)(fusion_fc2)
out = Dense(NUM_CLS, activation='softmax')(fusion_drop2)

In [25]:
model = Model(inputs=[hsi_input, lidar_input], outputs=[out])



sgd = SGD(lr=LEARNING_RATE, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 27, 27, 42)   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 41, 41, 14)   0                                            
__________________________________________________________________________________________________
hsi_conv1 (Conv2D)              (None, 27, 27, 500)  189500      input_1[0][0]                    
__________________________________________________________________________________________________
lidar_conv1 (Conv2D)            (None, 41, 41, 500)  63500       input_2[0][0]                    
__________________________________________________________________________________________________
batch_norm

In [26]:
history = model.fit([area_hsi_train_patches,area_lidar_train_patches], train_one_hot, batch_size=BATCH_SIZE, shuffle=True, epochs=300)

Epoch 1/300
2832/2832 [==============================] - 21s 7ms/step - loss: 2.8212 - acc: 0.1808
Epoch 2/300
2832/2832 [==============================] - 16s 6ms/step - loss: 2.2990 - acc: 0.2027
Epoch 3/300
2832/2832 [==============================] - 16s 6ms/step - loss: 2.2295 - acc: 0.2458
Epoch 4/300
2832/2832 [==============================] - 16s 6ms/step - loss: 2.1146 - acc: 0.2666
Epoch 5/300
2832/2832 [==============================] - 16s 6ms/step - loss: 2.0515 - acc: 0.2903
Epoch 6/300
2832/2832 [==============================] - 16s 6ms/step - loss: 1.9978 - acc: 0.3040
Epoch 7/300
2832/2832 [==============================] - 16s 6ms/step - loss: 1.9491 - acc: 0.3340
Epoch 8/300
2832/2832 [==============================] - 16s 6ms/step - loss: 1.9968 - acc: 0.2980
Epoch 9/300
2832/2832 [==============================] - 16s 6ms/step - loss: 1.8418 - acc: 0.3520
Epoch 10/300
2832/2832 [==============================] - 16s 6ms/step - loss: 1.8053 - acc: 0.3655
Epoch 11/

In [0]:
from operator import truediv

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [28]:
test_cls = test_labels - 1
prediction = model.predict([area_hsi_test_patches,area_lidar_test_patches]).argmax(axis=-1)



from sklearn import metrics, preprocessing

overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : ", each_acc)
print("Confusion matrix :", confusion_matrix)


Overall Accuracy of training sapmles :  0.5601377387882266
Average Accuracy of training samples :  0.6086397320497828
Kappa statistics of training samples :  0.5223330726966656
Each class accuracy of training samples :  [0.70089859 0.59138756 0.76421405 0.48892826 0.69059166 0.40073529
 0.49404762 0.90924956 0.60701754 0.446794   0.37185126 0.77065767
 0.46416382 0.91743119 0.51162791]
Confusion matrix : [[546  92   0  51   0   4   0   0   0   0   0   0   0   0  86]
 [231 618   0 145   0   0   2   0  48   0   0   0   1   0   0]
 [  0   0 457   0   0   0   0   0   0   0   0   0   0   0 141]
 [ 28  46   1 552   0  24 317   4   6  69   0   0   0  33  49]
 [ 18   0   0   4 712   0   0  20  34   0  60  83   0   0 100]
 [ 47   0   0  64   0 109   0   0  37   0   0   6   8   1   0]
 [  0  92   0  87   0   5 332  10 126   0   0   0  20   0   0]
 [  0   0   0   0   0   0   0 521   0   0   0  52   0   0   0]
 [  0  55   0  21   0   1 109   0 519  21  33  68  15   4   9]
 [  5   0   0   0 278   0